In [2]:
from gensim import corpora, models, similarities,matutils
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import numpy as np
import scipy.sparse as sp
import os
import time

In [3]:
def findVariationalParams(M,datapath,alpha,K):
    ''' 
    Function to determine the variational parameters
    Input: 
    M = DxV integer ndarray where 
           M(d,v) = Count of word v in document d
    alpha = K-vector of floats, shape = (K,) 
    beta = V-vector of floats, shape = (V,)
    
    Method: Use eq. 2,3,4 to determine outputs
    Refer to Blei et al.(2003) for definition of psi
    
    Output:
    eta: KxV float ndarray
    gamma: DxK float ndarray
    phi: DxVxK float ndarray 
         (returning a row_sparse D.V x K matrix phisp) 
    '''
    D,V = M.shape
    #K = alpha.shape[0]
    eta = np.ones ((K,V))
    gamma = np.ones((D,K))
    phi = np.ones((D,V,K))
    ################### CODE HERE #######################
    path = "lda-c/"
    param  = "param"
    cmd1 = "rm -r "+param
    cmd2 = "mkdir "+param
    cmd3 = path+"lda est "+str(alpha)+" "+ str(K) +" "+ path + \
          "settings.txt " + datapath + " random" + " ./" \
          + param
    os.system(cmd1)
    os.system(cmd2)
    os.system(cmd3)
    print("Reading phi")
    phi = np.loadtxt("param/final.phi").reshape(D,V,K)
    print("Reading gamma")
    gamma = np.loadtxt("param/final.gamma")
    print("Reading beta")
    beta = np.loadtxt("param/final.beta")
    print ("Building eta")
    for v in xrange(V):
        s=np.zeros((K,))
        for d in xrange(D):
            s+=M[d,v]*phi.T[:,v,d]
        eta[:,v] = beta[:,v] + s
    phisp = sp.csr_matrix(phi.reshape(D*V,K))
    #Use phisp preferably for operations
    #Usage phi[d,v,k]=phisp[d*V+v,k]
    #####################################################
    return eta,gamma,phisp,beta

In [1]:
def preprocessWords(inputPath,corpusfile,stopwords):
    '''
    Parses all files in the inputPath folder and
    returns the word matrix M:DxV of type ndarray(int32).
    Also stores the corpus in blei's LDA-C format as 
    corpusfile (corpusfile is a full path with filename).
    Input-specific stopwords also taken as array of strings
    '''
    porter = PorterStemmer()
    docs,docLen=[],0
    print("Reading data from %s"%inputPath)
    for path in inputPath:
        for filename in os.listdir(path):
            with open(path+filename,'r') as inp:
                print("Reading data from %s"%filename)
                f=inp.read()
                words=word_tokenize(f)
                words = [w.lower() for w in words]
                noPunc = [w.translate(None,string.punctuation)
                          for w in words]
                noEmp = [w for w in noPunc if w.isalpha()]
                noStop = [w for w in noEmp if not w
                          in stop_words]
                stemmed = [porter.stem(w) for w in noStop]
                stemmed = [w for w in stemmed if not w
                          in stop_words]
            docLen+=len(stemmed)
            docs.append(stemmed)
            #docs.append(noStop)
    D = len(docs)
    print ("Total Number of documents = %d"%D)
    print("Average words per document = %d"%(docLen/D))
    dcy = corpora.Dictionary(docs)
    V = len(dcy)
    print("Total vocabulary size = %d"%V)
    #dcy.save(os.path.join(TMP,'cong.dict'))
    corpus = [dcy.doc2bow(text) for text in docs]
    corpora.BleiCorpus.serialize(corpusfile,corpus)
    M = matutils.corpus2dense(corpus, num_terms=V, num_docs=D,
                              dtype=np.int32).T
    return M


In [ ]:
t0=time.time()
stop_words = stopwords.words('english')
stop_words += ['mr','would','say','lt', 'p', 'gt',
               'amp', 'nbsp','bill','speaker','us',
               'going','act','gentleman','gentlewoman',
               'chairman','nay','yea','thank']
pathnames = ['./convote_v1.1/data_stage_one/'+wor+'/'
             for wor in ['development_set','training_set']]
os.system("rm -r datafiles")
os.system("mkdir datafiles")
PTH = "./datafiles/congCorp.lda-c"
alpha = 0.1
K = 10
M = preprocessWords(pathnames,PTH,stopwords)
eta,gamma,phisp,beta=findVariationalParams(M,PTH,alpha,K)
t1=time.time()
print ("Time taken = %f"%(t1-t0))